# Classificação de Filmes usando suas Legendas

In [1]:
import re
import glob
from multiprocessing import Pool
import numpy as np
import pandas as pd
import nltk

## Pré-Processamento

Primeiro iremos definir algumas constantes que serão úteis para nossa filtragem de dados.

In [2]:
TITLE = re.compile("(\/)(.*?)(\.)")
HTML_TAG = re.compile("<.*?>")

STOP_WORDS = nltk.corpus.stopwords.words('portuguese')

Abrimos o arquivo `categories.tsv` que possui a informação do filepath para a legenda e o respectivo gênero do filme, e salvamos essa informação em um DataFrame do pandas. Podemos observar então as primeiras cinco entradas do nosso arquivo, e ver que nosso DataFrame possui duas colunas (filepath e genre).

In [3]:
subs = pd.read_csv("./categories.tsv", sep="\t", names=["filepath", "genre"])
subs.head(5)

,filepath,genre
0,10000 B.C.(2008).XViD-PreVaill.br.srt,Ação
1,127 Hours (2010).BDRip.Larceny.br.srt,Aventura
2,12 Rounds.DVDRip.aXXo.br.srt,Crime
3,15 Minutes(2001).br.srt,Crime
4,17 Again.720p.REFiNED.br.srt,Comédia


### _Parsing_ Arquivos

Com a informação do nome do arquivo de cada legenda, fazemos então o parse do texto de cada legenda, removendo as marcações de tempo, e para evitar informações irrelevantes, removemos os três primeiros e ultimos blocos de legenda, já que em geral são utilizadas para creditar as pessoas responsaveis pela legenda, assim como definir estilos para as legendas.

Note que estamos adicionando as strings de legenda ao nosso DataFrame __subs__ que agora possui três colunas (filepath, genre e subtitle)

In [4]:
def canonize(text):
    text = HTML_TAG.sub("", text)
    return text

def parse_text(filepath):
    def parse_block(block):
        lines   = block.split('\n')
        txt     = ' '.join(lines[2:])
        txt     = canonize(txt)
        return txt
    
    # We don't consider the first and last three blocks, since usually they're credits for
    # the translators and/or style definition for the subtitles.
    with open(filepath, encoding="latin-1") as f:
        sub_file = f.read()
        sub_file = sub_file.strip().replace('\r', '').split('\n\n')[3:-3]
        lines = map(parse_block, sub_file)
        return ' '.join(lines).strip()

In [5]:
subtitle = []

for row in subs['filepath']:
    filepath = 'Legendas/' + row
    try:
        text = parse_text(filepath)
    except FileNotFoundError:
        text = None

    subtitle.append(text)

# add subtitle column to our subs df
subs['subtitle'] = subtitle

# remove rows with NAs
subs = subs.dropna()

subs.head(5)

,filepath,genre,subtitle
0,10000 B.C.(2008).XViD-PreVaill.br.srt,Ação,E será sussurrada aos quatro ventos das grande...
1,127 Hours (2010).BDRip.Larceny.br.srt,Aventura,mas pense no que vamos tocar. Por favor. Preci...
2,12 Rounds.DVDRip.aXXo.br.srt,Crime,"Revisão: Bozano, Nininha e Virtualnet. 00:00:..."
3,15 Minutes(2001).br.srt,Crime,-Não perca tempo. -Está bem Ouviu o que eu dis...
4,17 Again.720p.REFiNED.br.srt,Comédia,"O'Donnell, poupe-se para o jogo! Só estou aque..."


## Classificadores

### Matriz TF-IDF

Agora temos em mãos todos os dados que iremos precisarpara realizar as nossas análises e treinar nossos modelos de predição.

Vamos então definir o método responsavel por _tokenizar_ nossas legendas e então usando o mesmo criar um _tokenizer_ usando o algoritmo TF-IDF. Note que criamos dois tokenizers, um sem utilizar _stemming_ e outro usando _stemming_ dos tokens.

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer


def tokenize(text, stem=False):
    ''' Tokenizer
    
    receives text (string) and return list of tokenized text, can receive an extra parameter in order
    to stem the strings.
    '''
    sent_tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle')
    
    tokens = [word.lower() for sentence in sent_tokenizer.tokenize(text) \
              for word in nltk.word_tokenize(sentence) if word.isalpha()]
    if stem:
        stemmer = nltk.stem.RSLPStemmer()
        stems = [stemmer.stem(token) for token in tokens]
        return stems
    else:
        return tokens
    
def tokenize_and_stem(text):
    return tokenize(text, stem=True)

# define TF-IDF parameters (w/o stemming)
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                   min_df=0.2, stop_words=STOP_WORDS,
                                  use_idf=True, tokenizer=tokenize, ngram_range=(1,3))

# define TF-IDF parameters (w/ stemming)
tfidf_vectorizer_stemmed = TfidfVectorizer(max_df=0.8, max_features=200000,
                                   min_df=0.2, stop_words=STOP_WORDS,
                                  use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

Tendo as matrizes dos termos e documentos usando o algoritmo _TF-IDF_, podemos então treinar nos classificadores utilizando a matriz com os termos com e sem _stemming_.

In [7]:
%time tfidf_matrix = tfidf_vectorizer.fit_transform(subs['subtitle'])
print("shape of TF-IDF Matrix: ", tfidf_matrix.shape)

%time tfidf_matrix_stemmed = tfidf_vectorizer_stemmed.fit_transform(subs['subtitle'])
print("shape of TF-IDF Matrix (stemmed tokens): ", tfidf_matrix_stemmed.shape)

CPU times: user 2min 42s, sys: 3.66 s, total: 2min 46s
Wall time: 2min 53s
shape of TF-IDF Matrix:  (644, 1648)
CPU times: user 6min 12s, sys: 5.28 s, total: 6min 17s
Wall time: 6min 26s
shape of TF-IDF Matrix (stemmed tokens):  (644, 2027)


### K-Nearest neighbors

Podemos então observar que nosso modelo possui praticamente nenhuma diferença entre o nivel de predição, ambos acertam em média __45%__ dos gêneros de todos os filmes.

In [8]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(algorithm='auto')
%time knn_classifier = knn.fit(tfidf_matrix, subs['genre'])

print()
knn_stemmed = KNeighborsClassifier(algorithm='auto')
%time knn_stemmed_classifier = knn_stemmed.fit(tfidf_matrix_stemmed, subs['genre'])

CPU times: user 3.98 ms, sys: 6.97 ms, total: 10.9 ms
Wall time: 11.5 ms

CPU times: user 2.76 ms, sys: 799 µs, total: 3.56 ms
Wall time: 3.35 ms


In [9]:
print("KNN Score: ", knn_classifier.score(tfidf_matrix, subs['genre']))
print("KNN Score (Stemmed): ", knn_stemmed_classifier.score(tfidf_matrix_stemmed, subs['genre']))

KNN Score:  0.459627329193
KNN Score (Stemmed):  0.453416149068


### Support vector machines (SVMs)

Podemos extrapolar um pouco e testar um novo método de predição, nesse caso estou usando _Support vector machines (SVMs) mais especificamente iremos utilizar _SVM_ com um _kernel_ linear.

Têmos um grande salto de precisão utilizando _SVMs_, tanto usando stemming como não usando stemming, respectivamente temos _82.91%_ e _82.41%_ de acerto usando os nossos dados para verificação. Portanto um ganho significativo de aproximadamente _40%_.

In [10]:
from sklearn import svm

svc_linear = svm.SVC(kernel='linear')
%time svc_linear_classifier = svc_linear.fit(tfidf_matrix, subs['genre'])

svc_linear_stemmed = svm.SVC(kernel='linear')
%time svc_linear_stemmed_classifier = svc_linear_stemmed.fit(tfidf_matrix_stemmed, subs['genre'])

CPU times: user 4.6 s, sys: 58.7 ms, total: 4.66 s
Wall time: 5.34 s
CPU times: user 4.85 s, sys: 45.5 ms, total: 4.89 s
Wall time: 4.96 s


In [11]:
print('svc linear (w/o stemming): ', svc_linear_classifier.score(tfidf_matrix, subs['genre']))
print('svc linear (w/ stemming) : ', svc_linear_stemmed_classifier.score(tfidf_matrix_stemmed, subs['genre']))

svc linear (w/o stemming):  0.824534161491
svc linear (w/ stemming) :  0.829192546584


## Testando Queries

Com nossas queries abaixo, podemos rodar nós dois melhores modelos usando cada técnica, KNN sem usar stemming e SVM Linear usando stemming.

In [12]:
# Queries in the format (title, Subtitle)
queries = {
          "The Bourne Identity": "Bourne Identity The.720p.ESiR.br.srt",
          "Bruce Almighty": "Bruce.Almighty.XviD.MTT.srt",
          "Free Willy": "Lord of the Rings The Fellowship of the Ring The.DVDRip.SecretMyth.br.srt",
          "The Godfather": "Fantastic Four.DVDRip.br.srt",
          "Iron Man": "Iron Man.TS.KingBen.br.srt",
          "Platoon": "Platoon (1986).720p.br.srt",
          "Puss in Boots": "Puss in Boots (2011).BRRIP.3LT0N.br.srt",
          "Scarface": "Scarface.br.srt",
          "Pretty Woman": "Pretty Woman.720p.AVS720.br.srt",
          "Fast Food Nation": "Fast Food Nation(2006).br.srt",
          }


# titles = []
# genres = []
# predicted_genres = []

# for title, filepath in queries.items():
#     subs_entry = subs[subs['filepath'] == filepath]
    
#     querie_tfidf = tfidf_vectorizer.transform(subs_entry['subtitle'])
#     predicted = knn_classifier.predict(querie_tfidf)[0]
    
#     # populate columns
#     titles.append(title)
#     genres.append(subs_entry['genre'].to_string(index=False))
#     predicted_genres.append(predicted)
    

# df_results = pd.DataFrame({'title': titles, 'genres': genres, 'predicted': predicted_genres},
#                           columns=['title', 'genres', 'predicted'])

In [13]:
def test_queries(queries, classifier, vectorizer):
    titles = []
    genres = []
    predicted_genres = []

    for title, filepath in queries.items():
        subs_entry = subs[subs['filepath'] == filepath]

        querie_tfidf = vectorizer.transform(subs_entry['subtitle'])
        predicted = classifier.predict(querie_tfidf)[0]

        # populate columns
        titles.append(title)
        genres.append(subs_entry['genre'].to_string(index=False))
        predicted_genres.append(predicted)


    df_results = pd.DataFrame({'title': titles, 'genres': genres, 'predicted': predicted_genres},
                              columns=['title', 'genres', 'predicted'])

    return df_results

### Análise Resultados Usando KNN

- __Pretty Woman__: Foi enquadrado como uma comédia, porêm o gênero correto seria comédia romântica, eu creio que é uma predição que faz bastante sentido.
- __Iron Man__: Alêm das varias tiradas  cômicas caracteristicas de filmes da frânquia do Homem de Ferro, não consigo entender bem o que faria esse filme ser classificado como comédia.
- __Free Willy__: Acertou :)
- __Platoon__: Nunca assisti o filme e baseado em sinopses do IMDB não consigo entender bem porque o modelo o classificaria como Ficção Científica, alêm do fato que muitos filmes de Ficção Científica em geral são sobre guerra.
- __Scarface__: Acertou
- __Bruce Almighty__: Acertou
- __Fast Food Nation__: Classificar documentarios é algo bastante complicado eu creio, dado o fato que eles em geral não têm um tema comum.
- __Puss in Boots__: Embora seja um filme Infantil é um filme de aventura de certa forma, fazendo sentido ele ser classificado como uma aventura.
- __The Bourne Identity__: Não consigo entender bem o porque esse filme foi classificado como uma comédia.
- __The Godfather__: Acertou

In [14]:
test_queries(queries, knn_classifier, tfidf_vectorizer)

,title,genres,predicted
0,The Bourne Identity,Ação,Comédia
1,Fast Food Nation,Documentário,Comédia
2,Platoon,Ação,Ficção científica
3,Free Willy,Aventura,Aventura
4,The Godfather,Aventura,Aventura
5,Puss in Boots,Infantil,Aventura
6,Iron Man,Aventura,Comédia
7,Bruce Almighty,Comédia,Comédia
8,Scarface,Crime,Crime
9,Pretty Woman,Comédia Romântica,Comédia


### Análise Resultados Usando SVM

- __Pretty Woman__: Alêm dos momentos de drama caracteristicos de comédias românticas, não entendo o porque o modelo classificaria esse filme como um Drama.
- __Iron Man__: Acertou.
- __Free Willy__: Acertou.
- __Platoon__: Acertou.
- __Scarface__: Acertou.
- __Bruce Almighty__: Acertou
- __Fast Food Nation__: Novamente eu creio que seja bastante complicado para o nosso modelo conseguir predizer o gênero documentario dado a abrangência de temas entre eles.
- __Puss in Boots__: Acertou.
- __The Bourne Identity__: Acertou.
- __The Godfather__: Acertou

In [15]:
test_queries(queries, svc_linear_stemmed_classifier, tfidf_vectorizer_stemmed)

,title,genres,predicted
0,The Bourne Identity,Ação,Ação
1,Fast Food Nation,Documentário,Drama
2,Platoon,Ação,Ação
3,Free Willy,Aventura,Aventura
4,The Godfather,Aventura,Aventura
5,Puss in Boots,Infantil,Infantil
6,Iron Man,Aventura,Aventura
7,Bruce Almighty,Comédia,Comédia
8,Scarface,Crime,Crime
9,Pretty Woman,Comédia Romântica,Drama
